In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, models



In [ ]:
base_dir = "/content/drive/MyDrive/pestopia_tier1"


In [ ]:
img_height, img_width = 150, 150
batch_size = 32
epochs_per_run = 20


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = train_gen.num_classes
print("Number of classes:", num_classes)


Found 9973 images belonging to 21 classes.
Found 2481 images belonging to 21 classes.
Number of classes: 21


In [ ]:
model_dir = "/content/drive/MyDrive/models"
os.makedirs(model_dir, exist_ok=True)


In [ ]:
def identity_block(x, f):
    skip = x
    x = layers.Conv2D(f, (3,3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(f, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, skip])
    return layers.Activation('relu')(x)

def ResNet(shape, classes):
    inputs = layers.Input(shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    x = identity_block(x, 64)
    x = identity_block(x, 64)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

ResNetmodel = ResNet((img_height, img_width, 3), num_classes)
ResNetmodel.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

ResNetmodel.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 75, 75,    │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 38, 38,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 38, 38,    │     36,928 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 38, 38,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 38, 38,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 38, 38,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 38, 38,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 38, 38,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 38, 38,    │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 38, 38,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 38, 38,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 38, 38,    │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 38, 38,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 38, 38,    │          0 │ add_1[0][0]       │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ activation_1[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │     33,280 │ global_average_p

 Total params: 202,261 (790.08 KB)

 Trainable params: 201,749 (788.08 KB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
resnet_history = ResNetmodel.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs_per_run,
    callbacks=[ModelCheckpoint(
        filepath=os.path.join(model_dir, "ResNet_epoch_{epoch:02d}.h5"),
        save_freq="epoch",
        verbose=1
    )]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.3100 - loss: 2.3353 
Epoch 1: saving model to /content/drive/MyDrive/models/ResNet_epoch_01.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 8114s 26s/step - accuracy: 0.3101 - loss: 2.3352 - val_accuracy: 0.2406 - val_loss: 2.4315
Epoch 2/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.3694 - loss: 2.1471
Epoch 2: saving model to /content/drive/MyDrive/models/ResNet_epoch_02.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 187ms/step - accuracy: 0.3694 - loss: 2.1471 - val_accuracy: 0.2672 - val_loss: 2.6443
Epoch 3/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.3868 - loss: 2.0733
Epoch 3: saving model to /content/drive/MyDrive/models/ResNet_epoch_03.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 185ms/step - accuracy: 0.3868 - loss: 2.0733 - val_accuracy: 0.2620 - val_loss: 2.3749
Epoch 4/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.4019 - loss: 2.0148
Epoch 4: saving model to /content/drive/MyDrive/models/ResNet_epoch_04.h5


...

312/312 ━━━━━━━━━━━━━━━━━━━━ 59s 188ms/step - accuracy: 0.5610 - loss: 1.4311 - val_accuracy: 0.4369 - val_loss: 1.9258
Epoch 17/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.5667 - loss: 1.4105
Epoch 17: saving model to /content/drive/MyDrive/models/ResNet_epoch_17.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 186ms/step - accuracy: 0.5667 - loss: 1.4106 - val_accuracy: 0.4740 - val_loss: 1.9031
Epoch 18/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.5780 - loss: 1.3850
Epoch 18: saving model to /content/drive/MyDrive/models/ResNet_epoch_18.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 187ms/step - accuracy: 0.5780 - loss: 1.3850 - val_accuracy: 0.3954 - val_loss: 2.8548
Epoch 19/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.5819 - loss: 1.3452
Epoch 19: saving model to /content/drive/MyDrive/models/ResNet_epoch_19.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 186ms/step - accuracy: 0.5819 - loss: 1.3453 - val_accuracy: 0.4655 - val_loss: 1.8939
Epoch 20/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.6049 - loss: 1.2848
Epoch 20: saving model to /content/drive/MyDrive/models/ResNet_epoch_20.h5


312/312 ━━━━━━━━━━━━━━━━━━━━ 58s 187ms/step - accuracy: 0.6048 - loss: 1.2849 - val_accuracy: 0.5212 - val_loss: 1.6657
